
# 🧱 C5.2.1 — Construire des variables (Bloc 5)

**Objectif de la compétence (éliminatoire)** : à partir des avis texte, **construire des variables** pertinentes et reproductibles
pour la modélisation (sentiment binaire/émotions/ABSA), et produire un **jeu de données exploitable**.

Ce notebook :
- lit le **dataset normalisé** produit en C5.1.1 (`data/interim/amazon_electronics_normalized.parquet|csv`),
- applique des **prétraitements contrôlés**,
- génère plusieurs **familles de features** (TF‑IDF mots, TF‑IDF caractères, *hand‑crafted*),
- sauvegarde les **artefacts de vectorisation** et un **échantillon de features** réutilisable,
- documente les **choix** et pourquoi la compétence est **validée**.


## 🎙️ Discours rapide


> « Nous construisons d'abord des **features textuelles robustes** et peu coûteuses : **TF‑IDF mots** pour la sémantique directe
> et **TF‑IDF caractères** pour la morphologie/ponctuation (utile pour fautes/sarcasme).  
> Nous ajoutons quelques **indicateurs** (longueur, !, ? , MAJUSCULES) et conservons des **métadonnées** utiles (*verified*, *helpful*).  
> Les recettes sont **paramétrées** (ngram, min_df, max_features), **reproductibles** (seeds), et **sérialisées** (`joblib`).
> On produit un **jeu de données exploitable** (X, y) et les **artefacts** pour l'entraînement (C5.2.3). »


## ⚙️ 0) Config & chemins

In [1]:

from pathlib import Path

PARQUET_NORM = Path("data/interim/amazon_electronics_normalized.parquet")
CSV_NORM     = Path("data/interim/amazon_electronics_normalized.csv")

TEXT_COL   = "review_body"
LABEL_COL  = "star_rating"   # utilisé pour fabriquer y binaire >=4
DATE_COL   = "review_date"   # optionnel

# Sampler pour itérer vite (None = tout)
SAMPLE_N = 200_000   # tu peux augmenter/réduire selon ta RAM
RANDOM_STATE = 42

# Recette TF-IDF mots
WORD_MAX_FEATURES = 60_000
WORD_NGRAM_RANGE  = (1, 2)
WORD_MIN_DF       = 3

# Recette TF-IDF caractères (désactivable ici)
USE_CHAR_TFIDF    = True
CHAR_MAX_FEATURES = 120_000
CHAR_NGRAM_RANGE  = (3, 5)
CHAR_MIN_DF       = 3

# Dossiers sortie
ART_DIR = Path("models")
DATA_PROC_DIR = Path("data/processed")
ART_DIR.mkdir(parents=True, exist_ok=True)
DATA_PROC_DIR.mkdir(parents=True, exist_ok=True)

## 📥 1) Chargement du dataset normalisé

In [2]:

import pandas as pd

if PARQUET_NORM.exists():
    df = pd.read_parquet(PARQUET_NORM)
    print("✓ Chargé depuis :", PARQUET_NORM)
elif CSV_NORM.exists():
    df = pd.read_csv(CSV_NORM)
    print("✓ Chargé depuis :", CSV_NORM)
else:
    raise FileNotFoundError("Dataset normalisé introuvable. Exécute C5.1.1 (cellule de sauvegarde).")

# échantillonnage (option)
if SAMPLE_N is not None and len(df) > SAMPLE_N:
    df = df.sample(SAMPLE_N, random_state=RANDOM_STATE)

df[TEXT_COL] = df[TEXT_COL].astype(str)
df["review_len"] = df[TEXT_COL].str.len()
print("Shape:", df.shape)
df[[TEXT_COL, "review_len"]].head(3)

✓ Chargé depuis : data\interim\amazon_electronics_normalized.parquet
Shape: (200000, 12)


,review_body,review_len
145322,This Hub was clearly designed for Apple comput...,509
1110436,Works as advertised,19
301223,Good so far! Only had for 1 week. Clips for st...,70


## 🧼 2) Prétraitements contrôlés

In [3]:

import re

def basic_clean(s: str) -> str:
    s = s.lower()
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Exemple : ne pas supprimer la ponctuation ici (laissons TF‑IDF char l'exploiter)
df["text_clean"] = df[TEXT_COL].map(basic_clean)
df["word_count"] = df["text_clean"].str.split().str.len()
df["bang_count"] = df[TEXT_COL].str.count("!")
df["ques_count"] = df[TEXT_COL].str.count("\?")
df["upper_ratio"] = df[TEXT_COL].apply(lambda x: (sum(1 for ch in x if ch.isupper()) / max(1,len(x))))
df[["text_clean","word_count","bang_count","ques_count","upper_ratio"]].head(3)

,text_clean,word_count,bang_count,ques_count,upper_ratio
145322,this hub was clearly designed for apple comput...,92,0,0,0.043222
1110436,works as advertised,3,0,0,0.052632
301223,good so far! only had for 1 week. clips for st...,16,1,0,0.042857


## 🎯 3) Label binaire (≥4 = positif)

In [4]:

import pandas as pd
y = (pd.to_numeric(df[LABEL_COL], errors="coerce") >= 4).astype(int).values
print("Positives ratio:", y.mean())

Positives ratio: 0.761405


In [7]:
# 3) Statistiques de classes + weights pour l'entraînement
import numpy as np, json
from sklearn.utils.class_weight import compute_class_weight

classes = np.array([0, 1])
cw = compute_class_weight(class_weight="balanced", classes=classes, y=y)
class_weight = {int(c): float(w) for c, w in zip(classes, cw)}

print(f"Distribution: neg={((y==0).mean()):.2%} | pos={((y==1).mean()):.2%}")
print("class_weight:", class_weight)

# Vecteur de poids par échantillon (aligné sur y)
sample_weight = np.where(y == 1, class_weight[1], class_weight[0])
print("sample_weight shape:", sample_weight.shape, "| min/max:", sample_weight.min(), sample_weight.max())

Distribution: neg=23.86% | pos=76.14%
class_weight: {0: 2.0956013328024476, 1: 0.6566807415238933}
sample_weight shape: (200000,) | min/max: 0.6566807415238933 2.0956013328024476


## 🔤 4) TF‑IDF mots — vectorisation principale

In [13]:
# 4) TF-IDF mots — vectorisation principale (sans fonction picklée)
from sklearn.feature_extraction.text import TfidfVectorizer

vec_word = TfidfVectorizer(
    lowercase=False,                 # déjà en minuscules via text_clean
    stop_words="english",
    max_features=WORD_MAX_FEATURES,
    ngram_range=WORD_NGRAM_RANGE,
    min_df=WORD_MIN_DF,
)

X_word = vec_word.fit_transform(df["text_clean"])  # ✅ on utilise le texte nettoyé
X_word.shape

(200000, 60000)

## 🔡 5) (Option) TF‑IDF caractères — complément morpho/ponctuation

In [14]:
# 5) TF-IDF caractères — complément morpho/ponctuation (sans fonction picklée)
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse

if USE_CHAR_TFIDF:
    vec_char = TfidfVectorizer(
        analyzer="char",
        lowercase=False,                # déjà en minuscules via text_clean
        max_features=CHAR_MAX_FEATURES,
        ngram_range=CHAR_NGRAM_RANGE,
        min_df=CHAR_MIN_DF,
    )
    X_char = vec_char.fit_transform(df["text_clean"])  # ✅ texte nettoyé
    # Concaténer horizontalement (word | char)
    X_all = sparse.hstack([X_word, X_char], format="csr")
    print("X_all shape:", X_all.shape, "(word + char)")
else:
    vec_char = None
    X_all = X_word
    print("X_all shape:", X_all.shape, "(word only)")

X_all shape: (200000, 180000) (word + char)


## 🧮 6) Features *hand‑crafted* (indicateurs simples)

In [15]:

import numpy as np
from scipy import sparse

# Metadonnées disponibles (optionnelles)
meta_cols = [c for c in ["word_count","review_len","bang_count","ques_count","upper_ratio",
                         "helpful_vote","verified_purchase"] if c in df.columns]
meta = df[meta_cols].copy() if meta_cols else None

if meta is not None:
    # Remplissage / standardisation légère
    for c in meta_cols:
        if meta[c].dtype == "bool":
            meta[c] = meta[c].astype(int)
        else:
            meta[c] = pd.to_numeric(meta[c], errors="coerce").fillna(0.0)
    M = sparse.csr_matrix(meta.values)
    X_all = sparse.hstack([X_all, M], format="csr")
    print("Ajout meta:", meta_cols, "| X_all:", X_all.shape)
else:
    print("Pas de colonnes meta ajoutées.")

Ajout meta: ['word_count', 'review_len', 'bang_count', 'ques_count', 'upper_ratio', 'helpful_vote', 'verified_purchase'] | X_all: (200000, 180007)


## 💾 7) Sauvegarde des artefacts de features

In [16]:
from joblib import dump
import time, json

stamp = time.strftime("%Y%m%d_%H%M%S")
art = {
    "vectorizer_word": vec_word,
    "vectorizer_char": vec_char,
    "feature_meta_cols": meta_cols,
    "created": stamp,
    "params": {
        "word_max_features": WORD_MAX_FEATURES,
        "word_ngram_range": WORD_NGRAM_RANGE,
        "word_min_df": WORD_MIN_DF,
        "use_char_tfidf": USE_CHAR_TFIDF,
        "char_max_features": CHAR_MAX_FEATURES,
        "char_ngram_range": CHAR_NGRAM_RANGE,
        "char_min_df": CHAR_MIN_DF,
    }
}
art_path = ART_DIR / f"features_tfidf_{stamp}.joblib"
dump(art, art_path)
print("Artefact TF-IDF →", art_path)

Artefact TF-IDF → models\features_tfidf_20250912_125553.joblib


In [17]:
# 7) Sauvegarde des poids et méta pour la suite (C5.2.3)
from joblib import dump
from pathlib import Path
import time, json

stamp2 = time.strftime("%Y%m%d_%H%M%S")
META_PATH = DATA_PROC_DIR / "binary_label_metadata.json"
W_VEC_PATH = DATA_PROC_DIR / "sample_weight_binary.joblib"
CW_JSON   = DATA_PROC_DIR / "class_weight.json"

meta = {
    "n_samples": int(len(y)),
    "prevalence_pos": float((y==1).mean()),
    "class_weight": {k: float(v) for k, v in class_weight.items()},
    "created": stamp2,
}

META_PATH.write_text(json.dumps(meta, indent=2), encoding="utf-8")
(Path(CW_JSON)).write_text(json.dumps(meta["class_weight"], indent=2), encoding="utf-8")
dump(sample_weight, W_VEC_PATH)

print("Écrit →", META_PATH)
print("Écrit →", CW_JSON)
print("Écrit →", W_VEC_PATH)

Écrit → data\processed\binary_label_metadata.json
Écrit → data\processed\class_weight.json
Écrit → data\processed\sample_weight_binary.joblib


## 📦 8) Export d’un échantillon (X, y) exploitable

In [18]:

from scipy import sparse
from joblib import dump

X_path = DATA_PROC_DIR / "X_tfidf_sample.npz"
y_path = DATA_PROC_DIR / "y_binary_sample.joblib"

# On exporte un échantillon compact (pour entraînements rapides / démos)
sample_k = min(X_all.shape[0], 120_000)
X_sample = X_all[:sample_k]
y_sample = y[:sample_k]

sparse.save_npz(X_path, X_sample, compressed=True)
dump(y_sample, y_path)
print("Écrit :", X_path, "et", y_path, "| shape:", X_sample.shape)

Écrit : data\processed\X_tfidf_sample.npz et data\processed\y_binary_sample.joblib | shape: (120000, 180007)


## 🔍 9) Aperçu top n‑grammes (sanity check)

In [19]:

import numpy as np
vocab = {i:t for t,i in vec_word.vocabulary_.items()}
idf = vec_word.idf_
top_idx = np.argsort(idf)[:20]  # plus fréquents (idf bas)
[(vocab[i], float(idf[i])) for i in top_idx]

[('great', 2.4375084485141327),
 ('good', 2.8355909727561093),
 ('works', 2.951440385085651),
 ('use', 2.983627400496733),
 ('just', 3.082731932658865),
 ('br', 3.0985885871957026),
 ('like', 3.1066944161604733),
 ('product', 3.1845848597906405),
 ('quality', 3.264090793830936),
 ('easy', 3.2964589270513054),
 ('work', 3.3317614702818257),
 ('price', 3.435378904245009),
 ('love', 3.440186880995992),
 ('sound', 3.5449164637125294),
 ('br br', 3.5450438928302552),
 ('time', 3.5651893850658634),
 ('bought', 3.5656445956085965),
 ('really', 3.635688028963809),
 ('don', 3.7053837725345513),
 ('case', 3.7701965976275598)]


## ✅ Rappel à la compétence & validation

**Compétence :** *C5.2.1 — Construire des variables* (**éliminatoire**)  
**Pourquoi c’est validé :**
- Les **recettes de features** sont **claires et paramétrées** (TF‑IDF mots/char, meta), reproductibles.
- Les **artefacts** sont **sauvegardés** (`models/features_tfidf_*.joblib`) et un **jeu de données exploitable** est généré
  (`data/processed/X_tfidf_sample.npz`, `y_binary_sample.joblib`).
- Les **choix** sont justifiés (mots = sémantique, caractères = orthographe/ponctuation/sarcasme léger, indicateurs = style/qualité).
- La démarche est **scalable** (SAMPLE_N contrôlable) et **compatible** avec C5.2.3 (entraînement du modèle).